In [ ]:
import pickle, os
from pyecharts import options as opts
from pyecharts.charts import Kline, Line, Grid, Bar
import pandas as pd
import akshare as ak
from log import logger
from fetch_stock_data import get_data_from_all


# 读取temp_data/stock_data_20210701_20230328.pkl
with open('../temp_data/stock_data_20210701_20230328.pkl', 'rb') as f:
    stock_data = pickle.load(f)


In [ ]:
# 读取股票数据
# df = stock_data[('600519', '贵州茅台')]


def kline_compare_base(stock_df,axis_index = 0):
    # 将数据转换为列表格式
    data = stock_df[['open', 'close', 'low', 'high']].values.tolist()
    dates = stock_df.index.tolist() # date type:str, like '2021-07-01'

    # 绘制 K 线图
    kline = (
        Kline(init_opts=opts.InitOpts(bg_color="#000000"))

        

        .add_xaxis(dates)
        .add_yaxis(
            series_name = "K线图",
            y_axis = data,
            xaxis_index=axis_index,
            yaxis_index=axis_index,
            itemstyle_opts=opts.ItemStyleOpts(
                color="#ec0000",
                color0="#00da3c",
                border_color="#8A0000",
                border_color0="#008F28",
            ),
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(title="双品种组合"),
            xaxis_opts=opts.AxisOpts(
                # type_='category',
                is_scale=True,
                splitline_opts=opts.SplitLineOpts(is_show=False)
            ),
            yaxis_opts=opts.AxisOpts(
                is_scale=True,
                splitline_opts=opts.SplitLineOpts(is_show=False)
            ),
            legend_opts=opts.LegendOpts(
                is_show=True, textstyle_opts=opts.TextStyleOpts(color='#DCDCDC'))
        )
        .set_series_opts(
            # label_opts=opts.LabelOpts(is_show=False),  # 去掉line上的标签(数值)
            label_opts=opts.LabelOpts(is_show=False)
        )
    )

    # 绘制均线图
    # 计算均线数据
    ma5 = stock_df['close'].rolling(window=5).mean().tolist()
    ma10 = stock_df['close'].rolling(window=10).mean().tolist()
    ma20 = stock_df['close'].rolling(window=20).mean().tolist()


    line = (
        Line()
        .set_series_opts(
            label_opts=opts.LabelOpts(is_show=False),  # 去掉line上的标签(数值)
        )
        .add_xaxis(dates)
        .add_yaxis("MA5", ma5, symbol='none')  # symbol='none'去掉line上的圆点
        .add_yaxis("MA10", ma10, symbol='none')
        .add_yaxis("MA20", ma20, symbol='none')
    )

    # 将 K 线图和均线图组合在一起
    k_ma  = kline.overlap(line)
    return  k_ma

def stock_compare_vol(stock_df,axis_index = 0):
    data = stock_df['vol'].astype(int).values.tolist() # pyecharts的数据类型必须是int,如果是float,则会无法显示
    dates = stock_df.index.tolist() # date type:str, like '2021-07-01'
    bar = (
        Bar()

        .set_global_opts(
            xaxis_opts=opts.AxisOpts(
                # type_='category',
                is_scale=True,
                splitline_opts=opts.SplitLineOpts(is_show=False),
                axislabel_opts=opts.LabelOpts(is_show=False)
            ),
            yaxis_opts=opts.AxisOpts(
                is_scale=True,
                splitline_opts=opts.SplitLineOpts(is_show=False)
            ),
            legend_opts=opts.LegendOpts(
                is_show=False, textstyle_opts=opts.TextStyleOpts(color='#DCDCDC'))
        )

        .add_xaxis(dates)
        .add_yaxis(
            series_name = "成交量",
            y_axis = data,
            xaxis_index=axis_index,
            yaxis_index=axis_index,
            itemstyle_opts=opts.ItemStyleOpts(
                color="#ec0000",
                # color0="#00da3c",
                # border_color="#8A0000",
                # border_color0="#008F28",
            ),
        )
        .set_series_opts(
            label_opts=opts.LabelOpts(is_show=False)
        )
    )

    # 绘制均线图
    # 计算均线数据
    # ma5 = stock_df['close'].rolling(window=5).mean().tolist()
    # ma10 = stock_df['close'].rolling(window=10).mean().tolist()
    # ma20 = stock_df['close'].rolling(window=20).mean().tolist()


    # line = (
    #     Line()
    #     .set_series_opts(
    #         label_opts=opts.LabelOpts(is_show=False),  # 去掉line上的标签(数值)
    #     )
    #     .add_xaxis(dates)
    #     .add_yaxis("MA5", ma5, symbol='none')  # symbol='none'去掉line上的圆点
    #     .add_yaxis("MA10", ma10, symbol='none')
    #     .add_yaxis("MA20", ma20, symbol='none')
    # )

    # 将 K 线图和均线图组合在一起
    # k_ma  = bar.overlap(line)
    return  bar


In [ ]:
def kline_compare(df_top,df_bottom,start_value,end_value):
    
    kline_top = kline_compare_base(df_top,axis_index=0)
    kline_bottom = kline_compare_base(df_bottom,axis_index=1)
    kline_top_vol = stock_compare_vol(df_top,axis_index=2)
    kline_top_bottom = stock_compare_vol(df_bottom,axis_index=3)

    kline_top.set_global_opts(datazoom_opts=[
        opts.DataZoomOpts(is_show=False,start_value=start_value, end_value=end_value, range_start=None,range_end=None, xaxis_index=[0,0], pos_top="97%",pos_bottom="1%"), 
        opts.DataZoomOpts(is_show=True,start_value=start_value, end_value=end_value, range_start=None,range_end=None, xaxis_index=[0,1], pos_top="97%", pos_bottom="1%"),
        opts.DataZoomOpts(is_show=False,start_value=start_value, end_value=end_value, range_start=None,range_end=None, xaxis_index=[0,2], pos_top="97%", pos_bottom="1%"),
        opts.DataZoomOpts(is_show=False,start_value=start_value, end_value=end_value, range_start=None,range_end=None, xaxis_index=[0,3], pos_top="97%", pos_bottom="1%"),
        ],
        # xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(is_show=False)),
    )
    
    grid = Grid(init_opts=opts.InitOpts(width="1600px", height="800px"))
    
    grid.add(kline_top, grid_opts=opts.GridOpts(pos_top="0%", pos_bottom="60%"))
    grid.add(kline_top_vol, grid_opts=opts.GridOpts(pos_top="40%", pos_bottom="50%"))
    grid.add(kline_bottom, grid_opts=opts.GridOpts(pos_top="50%", pos_bottom="10%"))
    grid.add(kline_top_bottom, grid_opts=opts.GridOpts(pos_top="90%", pos_bottom="0%"))

    return grid


df_top=stock_data[('600519', '贵州茅台')]
# df_top=get_data_from_all('600519',stock_data)
df_bottom = stock_data[('zs.000001', '上证指数')]
kline_compare(df_top,df_bottom,'2021-08-06','2022-03-29').render_notebook()

In [189]:
# read temp_data/all_yjbb.pkl
with open('../temp_data/all_yjbb.pkl', 'rb') as f:
    all_yjbb = pickle.load(f)
all_yjbb

bb_data = all_yjbb[all_yjbb['股票代码']=='600519'].sort_values(by='最新公告日期',ascending=True)

# read temp_data/all_data.pkl
with open('../temp_data/stock_data_20210701_20230328.pkl', 'rb') as f:
    all_data = pickle.load(f)
k_data = all_data[('600519', '贵州茅台')]

k_data.reset_index(inplace=True)
logger.debug(len(bb_data))
bb_data.drop_duplicates(subset=['最新公告日期'], keep='first', inplace=True)
logger.debug(len(bb_data))

k_data['date'] = pd.to_datetime(k_data['date'])
bb_data['最新公告日期'] = pd.to_datetime(bb_data['最新公告日期'])

[2023-03-30 16:39:12,721] xoyo_stock_tools:DEBUG: 31
[2023-03-30 16:39:12,722] xoyo_stock_tools:DEBUG: 27


In [210]:
merged_df = pd.merge_asof(bb_data,k_data, left_on='最新公告日期', right_on='date', direction='backward')
# 清晰merged_df中date1为None 的数据
merged_df = merged_df[merged_df['date'].notnull()]
bb_df = k_data.merge(
    merged_df.iloc[:, 1:17], how='left', left_on='date', right_on='date')
bb_df[bb_df['最新公告日期'].notnull()]


,date,open,close,high,low,vol,amount,swing,pct_chg,change,...,营业收入-季度环比增长,净利润-净利润,净利润-同比增长,净利润-季度环比增长,每股净资产,净资产收益率,每股经营现金流量,销售毛利率,所处行业,最新公告日期
57,2021-03-31,2045.10,2009.00,2046.02,2000.00,37154.0,7.489086e+09,2.24,-2.29,-47.05,...,13.4746,4.120647e+10,17.05,2.3590,108.271444,33.09,35.990043,91.302761,酿酒行业,2021-03-31
76,2021-04-28,2010.00,2042.00,2047.50,2003.00,81343.0,1.646357e+10,2.12,-2.50,-52.44,...,-0.1862,1.309377e+10,16.69,21.7842,118.694734,9.18,1.833397,91.672155,酿酒行业,2021-04-28
139,2021-07-30,1725.00,1678.99,1725.00,1657.76,77736.0,1.310972e+10,3.84,-4.05,-70.80,...,-19.6162,2.260166e+10,13.29,-27.3862,109.239591,15.34,10.046956,91.462828,酿酒行业,2021-07-31
192,2021-10-22,1855.00,1901.00,1921.01,1854.00,35963.0,6.794850e+09,3.63,3.09,57.00,...,17.7255,3.382710e+10,11.07,18.0646,118.175862,23.20,19.989689,91.325000,酿酒行业,2021-10-23
300,2022-03-31,1720.00,1719.00,1738.60,1708.80,25174.0,4.338023e+09,1.72,-0.64,-11.10,...,18.7041,4.669729e+10,13.33,14.6518,128.421444,31.41,41.131316,91.409188,酿酒行业,2022-03-31
317,2022-04-27,1767.12,1794.92,1810.00,1767.12,57402.0,1.028989e+10,2.48,3.60,62.44,...,-1.2444,1.395446e+10,6.57,8.4248,139.527590,8.29,-1.182053,91.676284,酿酒行业,2022-04-27
383,2022-08-03,1889.99,1885.00,1904.00,1877.00,24598.0,4.642442e+09,1.44,0.27,5.02,...,-19.2693,2.465399e+10,9.08,-23.3254,128.751834,14.20,17.289849,91.378421,酿酒行业,2022-08-03
430,2022-10-17,1726.00,1718.50,1727.99,1697.80,33550.0,5.740147e+09,1.74,-1.10,-19.11,...,16.2190,3.726617e+10,10.17,17.8762,138.790564,21.68,29.256776,91.191404,酿酒行业,2022-10-17


In [226]:
import pyecharts.options as opts
from pyecharts.charts import Bar, Line
import numpy as np

"""
Gallery 使用 pyecharts 1.0.0
参考地址: https://echarts.apache.org/examples/editor.html?c=multiple-y-axis

目前无法实现的功能:

1、暂无
"""

colors = ["#5793f3", "#d14a61", "#675bba"]
x_data = bb_df['date'].tolist()
legend_list = ["蒸发量", "降水量", "平均温度"]
evaporation_capacity = np.nan_to_num(bb_df['净利润-净利润']).astype(int).tolist()
rainfall_capacity = np.nan_to_num(bb_df['营业收入-季度环比增长']).astype(int).tolist()
average_temperature = np.nan_to_num(bb_df['净利润-同比增长']).astype(int).tolist()

bar = (
    Bar(init_opts=opts.InitOpts(width="1260px", height="720px"))
    .add_xaxis(xaxis_data=x_data)
    .add_yaxis(
        series_name="蒸发量", y_axis=evaporation_capacity, yaxis_index=0, color=colors[1]
    )
    .add_yaxis(
        series_name="降水量", y_axis=rainfall_capacity, yaxis_index=1, color=colors[0]
    )
    .extend_axis(
        yaxis=opts.AxisOpts(
            name="蒸发量",
            type_="value",
            min_=0,
            max_=250,
            position="right",
            axisline_opts=opts.AxisLineOpts(
                linestyle_opts=opts.LineStyleOpts(color=colors[1])
            ),
            axislabel_opts=opts.LabelOpts(formatter="{value} ml"),
        )
    )
    .extend_axis(
        yaxis=opts.AxisOpts(
            type_="value",
            name="温度",
            min_=0,
            max_=25,
            position="left",
            axisline_opts=opts.AxisLineOpts(
                linestyle_opts=opts.LineStyleOpts(color=colors[2])
            ),
            axislabel_opts=opts.LabelOpts(formatter="{value} °C"),
            splitline_opts=opts.SplitLineOpts(
                is_show=True, linestyle_opts=opts.LineStyleOpts(opacity=1)
            ),
        )
    )
    .set_global_opts(
        yaxis_opts=opts.AxisOpts(
            type_="value",
            name="降水量",
            min_=0,
            max_=250,
            position="right",
            offset=80,
            axisline_opts=opts.AxisLineOpts(
                linestyle_opts=opts.LineStyleOpts(color=colors[0])
            ),
            axislabel_opts=opts.LabelOpts(formatter="{value} ml"),
        ),
        tooltip_opts=opts.TooltipOpts(
            trigger="axis", axis_pointer_type="cross"),
    )
)

line = (
    Line()
    .add_xaxis(xaxis_data=x_data)
    .add_yaxis(
        series_name="平均温度", y_axis=average_temperature, yaxis_index=2, color=colors[2]
    )
)

bar.overlap(line).render_notebook()


C:\Users\xoyoj\AppData\Local\Temp\ipykernel_5652\2582016035.py:17: RuntimeWarning: invalid value encountered in cast
  evaporation_capacity = np.nan_to_num(bb_df['净利润-净利润']).astype(int).tolist()


AttributeError: module 'numpy' has no attribute 'nan_tonum'

In [223]:
import numpy as np
from pyecharts.charts import Bar

# 创建一个包含NaN值的NumPy数组
data = np.array([1, 2, np.nan, 4, 5])

# 将NaN值替换为0
data = np.nan_to_num(data)
data = data.astype(int).tolist()
print(data)
# data = [1,2,0,4,5]

# 创建一个柱状图，并添加数据
bar = Bar()
bar.add_xaxis(['A', 'B', 'C', 'D', 'E'])
bar.add_yaxis('数据', data)
bar.render_notebook()  


[1, 2, 0, 4, 5]
